# 函数装饰器和闭包

# 装饰器的基础知识

严格来说，装饰器只是语法糖。方便元编程（在运行时改变程序的行为）

两大作用：
- 把被修饰函数替换成其他函数
- 在加载模块时立即执行

In [3]:
# 一个简单的例子
# 使用装饰器将返回的函数替换
# 装饰器就是一个以被修饰函数为参数的函数

def deco(func):
    # 在装饰器函数内创建另一个函数并返回
    def inner():
        print("inner() is running!")
    return inner

@deco
def target():
    print("target() is running")

# 执行被修饰函数
target()

inner() is running!


In [5]:
# 将函数作为对象

target  # target成为函数inner的引用

<function __main__.deco.<locals>.inner()>

# Python何时执行装饰器

装饰器在导入模块时就会执行。

python中提到的导入时和运行时，前者指的就是import的时候就会运行的代码。

In [1]:
import registration

running register (<function f1 at 0x7fdc99060310>)
running register (<function f2 at 0x7fdc9aeb55e0>)


### 装饰器函数在定义函数之后立即执行。因此在导入时也会即刻执行，即使我们没有调用任何模块内的函数。


## 实际应用时两个特殊的地方：
### 1 装饰器函数和被装饰的函数在同一个模块中定义，但现实中经常是装饰器函数在一个模块中定义，然后应用的其他模块的函数上。
### 2 示例中装饰器函数返回了输入的函数，但现实中往往会在装饰器中定义一个新的函数并返回。

# 使用装饰器改进“策略”模式

In [6]:
from collections import namedtuple

Customer = namedtuple('Customer','name fidelity')

class LineItem:
    '''每种产品信息，产品名，数量，单价等'''
    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price

    def total(self):
        return self.price * self.quantity


class Order:
    '''账单'''
    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion

    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total

    def due(self):
        if self.promotion is None:
            discount = 0.0
        else:
            discount = self.promotion(order = self) # 此处promotion变量已经是一个函数了，直接执行括号运算符就行了。
        return self.total() - discount

    def __repr__(self):
        fm = "<Order total:{:.2f} due:{:.2f}>"
        return fm.format(self.total(),self.due())

# 全局变量，用来存储所有策略函数
promo = []

# 创建一个修饰器函数，将所有策略函数注册到全局变量中。
def promotion(func):
    promo.append(func)
    return func

@promotion
def fidelity_promo(order):
    '''满1000积分提供5%的折扣'''
    return order.total() * 0.05 if order.customer.fidelity >= 1000.0 else 0.0

@promotion
def bulkitem_promo(order):
    '''单个商品数量为20或以上时提供10%的折扣'''
    d = 0.0
    for item in order.cart:
        if item.quantity >= 20 :
            d += item.total() * 0.1
    return d

@promotion
def largeorder_promo(order):
    '''订单中达到或超过10种商品时提供7%的折扣'''
    distinct_item = { item.product for item in order.cart } #使用了集，集内不能有重复元素
    if len(distinct_item) >= 10 :
        return order.total() * 0.07
    return 0.0

def best_promo(order):
    '''返回最大优惠方案'''
    return max([pr(order) for pr in promo ])


In [9]:
joe = Customer('John Doe', 0)
ann = Customer('Ann Smith', 1100)
cart = [LineItem('banana', 4, 0.5),
        LineItem('apple', 15,1.5),
        LineItem('watermelon', 30, 5.0)]


Order(ann, cart, best_promo)

<Order total:174.50 due:159.50>

# 变量作用域规则

In [17]:
# 一个正常的例子 我们故意不去定义全局变量b
def f1(a):
    print(a)
    print(b)

f1(3)

3
9


In [18]:
# 一个不太好理解的例子：
b = 6
def f2(a):
    print(a)
    print(b)
    b = 9

f2(3)

3


UnboundLocalError: local variable 'b' referenced before assignment

上面例子返回的错误提示是局部变量b在定义全就被引用了。而忽视了我们之前定义的全局变量b。

这是python的一个语言设计：

**python不要求声明变量，但是假定在函数定义体中赋值的变量是局部变量。**

In [20]:
# 使用global关键字来声明变量即可：
b = 6
def f3(a):
    global b
    print(a)
    print(b)
    b = 9

f3(3)


3
6


In [21]:
b

9

In [22]:
from dis import dis
dis(f2)

  4           0 LOAD_GLOBAL              0 (print)
              2 LOAD_FAST                0 (a)
              4 CALL_FUNCTION            1
              6 POP_TOP

  5           8 LOAD_GLOBAL              0 (print)
             10 LOAD_FAST                1 (b)
             12 CALL_FUNCTION            1
             14 POP_TOP

  6          16 LOAD_CONST               1 (9)
             18 STORE_FAST               1 (b)
             20 LOAD_CONST               0 (None)
             22 RETURN_VALUE


In [23]:
dis(f3)

  5           0 LOAD_GLOBAL              0 (print)
              2 LOAD_FAST                0 (a)
              4 CALL_FUNCTION            1
              6 POP_TOP

  6           8 LOAD_GLOBAL              0 (print)
             10 LOAD_GLOBAL              1 (b)
             12 CALL_FUNCTION            1
             14 POP_TOP

  7          16 LOAD_CONST               1 (9)
             18 STORE_GLOBAL             1 (b)
             20 LOAD_CONST               0 (None)
             22 RETURN_VALUE


# 闭包

### 闭包指延伸了作用域的函数，其中包含函数定义体中引用，但不在定义体中定义的非全局变量。

In [28]:
# 使用类实现
class make_averager_c():
    def __init__(self):
        self._series = []

    def __call__(self, value):
        self._series.append(value)
        total = sum(self._series)
        return total/len(self._series)

In [29]:
avg_c = make_averager_c()
avg_c(10)

10.0

In [30]:
avg_c(15)

12.5

In [31]:
avg_c(20)

15.0

In [24]:
# 使用函数实现

def make_averager():
    series = []

    def averager(new_value):
        series.append(new_value)
        total = sum(series)
        return total/len(series)
    
    return averager

In [25]:
avg = make_averager()
avg(10)

10.0

In [26]:
avg(15)

12.5

In [27]:
avg(20)

15.0

In [32]:
# 两条曲线之间的范围即为闭包
# series 为自由变量，而非局部变量

def make_averager():
    #------------------------------
    series = []

    def averager(new_value):
        series.append(new_value)
        total = sum(series)
        return total/len(series)
    #-----------------------------
    return averager

In [41]:
avg = make_averager()
avg(10)

10.0

In [42]:
avg(15)

12.5

In [43]:
avg(20)

15.0

In [44]:
avg.__code__.co_varnames #局部变量

('new_value', 'total')

In [45]:
avg.__code__.co_freevars #自由变量

('series',)

In [46]:
avg.__closure__

(<cell at 0x7fdc99b71d60: list object at 0x7fdc9a72ed80>,)

In [47]:
avg.__closure__[0].cell_contents

[10, 15, 20]

### 闭包是一种函数，它会保留定义函数时存在的自由变量的绑定，这样调用函数时，虽然定义作用域不可用了，但是仍能使用那些绑定。

# nonlocal声明

In [52]:
# 继续优化上面的求平均函数：

def make_averager2():
    count = 0
    total = 0
    def averager2(new_value):
        total += new_value
        count += 1
        return total/count
    return averager2

In [53]:
a = make_averager2()

a(10)

UnboundLocalError: local variable 'total' referenced before assignment

**这里之所以会出现错误，而且提示total成为了局部变量，而不是自由变量，那是应为total是一个不可变类型。**

不可变类型（数字，字符串，元组）其实只能读，不能写。一旦写了，其实是python创建了一个新的变量。

而上面的例子中，series是数组，是可变类型。也没有直接赋值给它，而是调用了append函数。因此上面的例子没有问题。

解决办法： 使用**nonlocal**关键字来声明一下自由变量。


In [54]:
# 使用nonlocal来声明不可变变量为自由变量！

def make_averager3():
    count = 0
    total = 0
    def averager3(new_value):
        nonlocal count,total # 在调用和赋值的函数中声明
        total += new_value
        count += 1
        return total/count
    return averager3

In [55]:
    c = make_averager3()
    c(10)

10.0

In [56]:
c(15)

12.5

In [57]:
c(20)

15.0